# Noisy-And and Noisy Data MIL

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from noisy_and import NoisyAnd
import mil_benchmarks

N_CLASSES = 2
MAX_BAG = 7

utils.gpu_fix()

tf.random.set_seed(42)
print(tf.__version__)

In [ ]:
def define_model(shape=(28, 28, 1)):
  model = tf.keras.Sequential([
    layers.Input((MAX_BAG,) + shape),
    *utils.baseline_layers(shape, N_CLASSES),
    NoisyAnd(N_CLASSES),
  ])

  utils.compile(model, N_CLASSES)
  return model

print(define_model().summary())

In [ ]:
def eval_noise(dataset, input_shape):
  histories = []

  for i in range(10):
    noise = i / 20
    print(f'Noise: {noise:.2f}')
    history = utils.evaluate(define_model(input_shape), utils.preprocess(dataset, noise=noise))
    histories.append(history)
  
  return histories

cifar_histories = eval_noise(mil_benchmarks.standard.cifar10.load_0(onehot=False), (32, 32, 3))

In [ ]:
mnist_histories = eval_noise(mil_benchmarks.standard.mnist.load_0(onehot=False), (28, 28, 1))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

noises = np.arange(10) / 20

def get_val(hists, k):
  return np.max([np.vstack(h.history[k]).mean(axis=1) for h in hists], axis=1)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

axes[0].set_title('MNIST (Class 0)')
axes[0].plot(noises, get_val(mnist_histories, 'auc'))
axes[0].plot(noises, get_val(mnist_histories, 'val_auc'))
axes[0].set_xticks(noises)
axes[0].legend(['Train', 'Test'])
axes[0].tick_params(axis='x', labelrotation=45)

axes[1].plot(noises, get_val(cifar_histories, 'auc'))
axes[1].plot(noises, get_val(cifar_histories, 'val_auc'))
axes[1].legend(['Train', 'Test'])
axes[1].set_xticks(noises)
axes[1].set_title('CIFAR-10 (Class 5)')
axes[1].tick_params(axis='x', labelrotation=45)

plt.savefig('img/noisy-data.jpg', dpi=250)
plt.show()
plt.close()